In [29]:
import pandas as pd
import numpy as np
import sympy as sp
import math
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from operator import itemgetter

In [30]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

#### Test 1: Solve overdetermined system of equations with least squares solution

In [31]:
# Adjust data set
df_pre = pd.read_csv("BABA_2014_2018.csv")
df_pre.columns = df_pre.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
#df_pre[df_pre['Date'] == '9/19/14']
df_pre['Ones'] = 1
df_pre['RSI_yesterday'] = df_pre.RSI.shift(+1)
df_pre['MACD_yesterday'] = df_pre.MACD.shift(+1)
df_pre['Stochastic_RSI_yesterday'] = df_pre.Stochastic.shift(+1)
df_pre['Stochastic_SMA_yesterday'] = df_pre.Stochastic_SMA.shift(+1)
df = df_pre.loc[30:]
#df_pre = df_pre.loc[830:] # Looking only at 2018; include 'Dates' column to check
df = df[['Date', 'RSI_yesterday', 'MACD_yesterday', 'Stochastic_RSI_yesterday', 'Stochastic_SMA_yesterday', 'delta']]

In [32]:
# for index, row in df.iterrows():
#    print (row[2])

In [33]:
df.columns

Index(['Date', 'RSI_yesterday', 'MACD_yesterday', 'Stochastic_RSI_yesterday',
       'Stochastic_SMA_yesterday', 'delta'],
      dtype='object')

In [34]:
# Split data between training set and validation set
train, test = train_test_split(df, test_size=0.33)

In [35]:
# Create X and y matrices from features and outputs
features_train = train[['Date', 'RSI_yesterday', 'MACD_yesterday', 'Stochastic_RSI_yesterday', 'Stochastic_SMA_yesterday']].copy()
labels_train = train[['Date','delta']].copy()
features_test = test[['Date', 'RSI_yesterday', 'MACD_yesterday', 'Stochastic_RSI_yesterday', 'Stochastic_SMA_yesterday']].copy()
labels_test = test[['Date', 'delta']].copy()

# create numpy arrays from pandas dataframes to solve
# ft = features_train.values
# labels = labels_train.values
# ft_test = feaures_test.values
# labels_test = labels_test.values
print (labels_test.shape[0])

341


In [38]:
# pass in rows x and y
def distance(x,y,amtFeatures,pwr):
    dist = 0
    for i in range(1, amtFeatures+1):
        dist += pow(abs(x[i] - y[i]), pwr)
    return dist ** (1/float(pwr))

In [41]:
def find_kNN(data, labels, sample, k, amtFeatures, pwr):
    distances = []
    counts = {'Up': 0, 'Down': 0, 'Neutral': 0}
    
    for row in range(data.shape[0]):
        point = data.iloc[row]
        d = distance(x=point, y=sample, amtFeatures=amtFeatures, pwr=pwr)
        distances.append((point[0], d))
    
    distances.sort(key=itemgetter(1))

    NN = []
    
    for pt in range(k):        
        date = distances[pt][0]
        label_row = labels[labels['Date'] == date]
        NN.append((date, label_row.iloc[0][1]))
    
    for pt in NN:
        if pt[1] == 'Up':
            counts['Up'] =+ 1
        if pt[1] == 'Down':
            counts['Down'] =+ 1
        if pt[1] == 'Neutral':
            counts['Neutral'] =+ 1

    prediction = max(counts, key=counts.get)

#     print ('NN are', NN)
#     print (prediction)
    
    return prediction

In [42]:
find_kNN(data=features_train, labels=labels_train, sample=sample, k=5, amtFeatures=4, pwr=2)

'Up'

### Testing accuracy of kNN

In [47]:
# choose k (must be odd), Minkowski distance power, and amount of features
k = 7
pwr = 2
amtfeatures = 4

# counter
correct = 0

for i in range(features_test.shape[0]):
    prediction = find_kNN(data=features_train, labels=labels_train, sample=features_test.iloc[i], k=k, amtFeatures=amtfeatures, pwr=pwr)
    actual = labels_test.iloc[i][1]
    if prediction == actual:
        correct += 1
print ('Accuracy: ', correct, '/', features_test.shape[0], ', or ', correct/features_test.shape[0])

Accuracy:  166 / 341 , or  0.4868035190615836
